### Load Data From Parquet

In [1]:
import ta

def prep_regime_filter(regime_df, roc_col_name='regime_roc', mv_col_name='regime_ma',
                       close_name='regime_close',
                       ma_period=200,
                       roc_period=45):
    regime_df.loc[:, roc_col_name] = ta.momentum.ROCIndicator(regime_df.close_adj, n=roc_period).roc()
    regime_df.loc[:, mv_col_name] = regime_df.set_index(
        'date'
    ).close_adj.rolling('%dd' % ma_period, min_periods=1).mean().values
    regime_df.loc[:, close_name] = regime_df.close_adj
    return regime_df

In [2]:
import pandas as pd
import collections

Market = collections.namedtuple('Market', 'candle_data market_index sec_index')

market_metadata = {
    600: Market(candle_data='historical_pr_data_600_sec.parquet',
                market_index='S_and_P_600_index.parquet',
                sec_index='sector_index_1000.parquet'),
    400: Market(candle_data='historical_pr_data_400_sec.parquet',
                market_index='S_and_P_400_index.parquet',
                sec_index='sector_index_400.parquet'),
    500: Market(candle_data='historical_pr_data_sec.parquet',
                market_index='S_and_P_index.parquet',
                sec_index='sector_index.parquet')
}

candle_dfs = {
    x: pd.read_parquet(market_metadata[x].candle_data)
    for x in market_metadata
}

In [3]:
market_index_dfs = {
    x: pd.read_parquet(market_metadata[x].market_index)
    for x in market_metadata
}

In [4]:
sector_index_dfs = {
    x: pd.read_parquet(market_metadata[x].sec_index)
    for x in market_metadata
}

In [5]:
prep_market_regimes_dfs = {
    x: prep_regime_filter(market_index_dfs[x])
    for x in market_index_dfs
}

In [6]:
prep_sector_regime_dfs = {
    x: sector_index_dfs[x].groupby('sector').apply(lambda x: prep_regime_filter(x, roc_col_name='sector_roc',
                                                                     mv_col_name='sector_ma',
                                                                     close_name='sector_close'))
    for x in sector_index_dfs
}

In [7]:
candle_w_regime = {
    x: candle_dfs[x].merge(prep_market_regimes_dfs[x][['date', 'regime_roc', 'regime_close', 'regime_ma']],
                           on='date', how='left')
    for x in candle_dfs
}
candle_w_regime = {
    x: candle_w_regime[x].merge(prep_sector_regime_dfs[x][['date', 'sector', 'sector_roc', 'sector_close', 'sector_ma']],
                                on=['date', 'sector'], how='left')
    for x in candle_w_regime
}

In [8]:
df_norm = pd.concat((candle_w_regime[x] for x in candle_w_regime), axis=0, ignore_index=True)

In [9]:
df_norm = df_norm.drop_duplicates(subset=['date', 'symbol']).sort_values(['symbol', 'date']).reset_index(drop=True)

In [10]:
import ta
import numpy as np

def mean_atr(df, atr_period=14):
    df.loc[:, 'last_close'] = df.close_adj.shift(1)
    atr_high = np.maximum(df.high_adj, df.last_close)
    atr_low = np.minimum(df.high_adj, df.last_close)
    atr = atr_high - atr_low
    return atr, atr.ewm(span=atr_period, adjust=False).mean()

def get_indicators(sym_df, roc_period=45, sto_period=28, ma_period=200, atr_period=7):
    sto = ta.momentum.StochIndicator(high=sym_df.high_adj, low=sym_df.low_adj, close=sym_df.close_adj,
                                     n=sto_period)
    sym_df.loc[:, 'sto'] = sto.stoch_signal()
    atr_val, atr = mean_atr(sym_df, atr_period=atr_period) 
    sym_df.loc[:, 'atr'] = atr
    sym_df.loc[:, 'volatility'] = (sym_df.close_adj.diff() / sym_df.close_adj.shift(1)).abs().ewm(span=atr_period,
                                                                                                  adjust=False).mean()
    sym_df.loc[:, 'roc'] = ta.momentum.ROCIndicator(sym_df.close_adj, n=roc_period).roc()
    sym_df.loc[:, 'mv_avg'] = sym_df.set_index(
        'date'
    ).close_adj.rolling('%dd' % ma_period, min_periods=1).mean().values
    return sym_df

In [11]:
df_ind = df_norm.reset_index(drop=True).groupby('symbol').apply(get_indicators).reset_index(drop=True)

In [12]:
def get_entrances(df,
                  sto_oversold=20,
                  roc_change=0,
                  regime_roc_change=0,
                  volatility_thresh=0.05):
    df.loc[:, 'rsi_oversold_enter'] = 0
    df.loc[:, 'last_sto'] = df.sto.shift(1)
    df.loc[:, 'sto_roc'] = df.sto - df.last_sto
    df.loc[:, 'last_sto_roc'] = df.sto_roc.shift(1)
    df.loc[:, 'sto_oversold_enter'] = 0
    bool_index = df.sto <= sto_oversold
    bool_index &= df.roc > roc_change
    #bool_index &= (df.sto_roc > df.last_sto_roc)
    #bool_index &= (df.close_adj > df.mv_avg)
    #bool_index &= (df.regime_roc > regime_roc_change)
    bool_index &= (df.regime_close > df.regime_ma)
    #bool_index &= (df.sector_roc > regime_roc_change)
    bool_index &= (df.sector_close > df.sector_ma)
    bool_index &= (df.volatility < volatility_thresh)
    df.loc[bool_index, 'sto_oversold_enter'] = 1
    enter_cols = ['sto_oversold_enter']
    df.loc[:, 'entrances'] = df[enter_cols].sum(axis=1).clip(upper=1)
    return df

In [13]:
def get_exits(df, sto_overbought=80):
    df.loc[:, 'close_adj_last'] = df.close_adj.shift(1)
    df.loc[:, 'sto_overbought_exit'] = 0
    df.loc[df.sto >= sto_overbought, 'sto_overbought_exit'] = 1
    exit_cols = ['sto_overbought_exit']
    df.loc[:, 'exits'] = df[exit_cols].sum(axis=1).clip(upper=1)
    return df

In [14]:
df_enter_exit = df_ind.groupby('symbol').apply(lambda x: get_entrances(get_exits(x), roc_change=10))

In [15]:
def backtest(df):
    df.loc[:, 'next_open'] = df.open_adj.shift(-1)
    df.loc[df.index[-1], 'exits'] = 1
    df.loc[df.index[-1], 'entrances'] = 0
    enter_exit = df.loc[(df.exits == 1) | (df.entrances == 1)]
    enter_exit.loc[:, 'enter_exit_diff'] = enter_exit.entrances.diff()
    if enter_exit.shape[0] > 0:
        fi = enter_exit.index[0]
        fi_enter_exit = enter_exit.iloc[0]
        enter_exit.loc[fi, 'enter_exit_diff'] = fi_enter_exit.entrances - fi_enter_exit.exits
    enter_exit = enter_exit.loc[enter_exit.enter_exit_diff.isin([1, -1])]
    enters = enter_exit.loc[enter_exit.enter_exit_diff == 1]
    exits = enter_exit.loc[enter_exit.enter_exit_diff == -1]
    df.loc[:, 'actual_enter'] = 0
    df.loc[:, 'actual_exit'] = 0
    df.loc[enters.index, 'actual_enter'] = 1
    df.loc[exits.index, 'actual_exit'] = 1
    enter_exit.loc[:, 'index_col'] = enter_exit.index
    enter_exit.loc[:, 'next_signal'] = enter_exit.enter_exit_diff.shift(-1)
    enter_exit.loc[:, 'next_sig_next_open'] = enter_exit.next_open.shift(-1)
    enter_exit.loc[:, 'next_index'] = enter_exit.index_col.shift(-1)
    enter_exit = enter_exit.loc[(enter_exit.enter_exit_diff == 1) & (enter_exit.next_signal ==  -1)]
    enter_exit.loc[:, 'profit'] = enter_exit.next_sig_next_open - enter_exit.next_open
    df.loc[:, 'profit'] = 0
    df.loc[enter_exit.index, 'profit'] = enter_exit.profit
    df.loc[enter_exit.index, 'exit_price'] = enter_exit.next_sig_next_open
    df.loc[enter_exit.index, 'exit_index'] = enter_exit.next_index
    return df

In [16]:
from numba import jit
import numpy as np
import datetime as dt


def backtest_seq(df, stop_thresh=0.1, run_length=30, inv_price=10000,
                 prof_avg_offset=30, ewm_prof_offset=100, mv_avg=None, equity=None,
                 pickup_dt=dt.datetime(1990, 10, 18), mv_avg_ratio_thresh=.97):
    if mv_avg is None or equity is None:
        mv_avg = np.zeros(df.shape[0])
        equity = np.zeros(df.shape[0])
    df.loc[:, 'enter_exit_sig'] = df.entrances - df.exits
    df.loc[:, 'next_open'] = df.open_adj.shift(-1)
    profit, exit_profit, exit_index, actual_enter_exit, shares_arr, equity = backtest_numba(
        df.enter_exit_sig.values, df.close_adj.values,
        df.next_open.values, df.index.values, stop_thresh, run_length,
        inv_price, equity / mv_avg, (df['date'] >= pickup_dt).values,
        mv_avg_ratio_thresh
    )
    df.loc[:, 'profit'] = profit
    df.loc[:, 'exit_profit'] = exit_profit
    df.loc[:, 'cum_exit_profit'] = df.exit_profit.fillna(0).cumsum()
    df.loc[:, 'equity_curve'] = equity + df.cum_exit_profit
    bool_index = exit_index != -1
    df.loc[bool_index, 'exit_date'] = df.loc[exit_index[bool_index], 'date'].values
    df.loc[:, 'cum_profit'] = df.profit.fillna(0).cumsum()
    df.loc[:, 'purch_shares'] = shares_arr
    df.loc[:, 'norm_profit'] = profit / (df.next_open * shares_arr)
    df.loc[df.profit == 0, 'norm_profit'] = np.nan
    df.loc[:, 'avg_profit'] = df.norm_profit.rolling(prof_avg_offset, min_periods=1).mean()
    df.loc[:, 'avg_profit_std'] = df.norm_profit.rolling(prof_avg_offset, min_periods=1).std()
    df.loc[:, 'eavg_profit'] = df.avg_profit.ewm(ewm_prof_offset, ignore_na=True).mean()
    df.loc[:, 'avg_profit'] = df.avg_profit.fillna(0)
    df.loc[:, 'actual_enter_exit'] = actual_enter_exit
    df.loc[:, 'actual_enter'] = 0
    df.loc[:, 'actual_exit'] = 0
    df.loc[df.actual_enter_exit == 1, 'actual_enter'] = 1
    df.loc[df.actual_enter_exit == -1, 'actual_exit'] = 1
    df.loc[:, 'trade_count'] = df.actual_enter_exit.rolling(prof_avg_offset).sum()
    return df
    

#@jit(nopython=True)
def backtest_numba(enter_exit, close_price, open_price, df_index, stop_thresh,
                   run_length, inv_price, equity_signal, bool_date, mv_avg_ratio_thresh):
    in_trade = False
    n = len(enter_exit)
    actual_enter_exit = np.zeros(n)
    shares_arr = np.zeros(n)
    equity = np.zeros(n)
    profit = np.zeros(n)
    exit_profit = np.zeros(n)
    exit_index = np.zeros(n) - 1
    start_price = 0.0
    enter_price = 0.0
    top_price = start_price
    shares = 0
    for index in range(0, n):
        signal = enter_exit[index]
        equity_stop_signal = equity_signal[index] < mv_avg_ratio_thresh and bool_date[index] == True
        if in_trade and close_price[index] > top_price:
            top_price = close_price[index]
        if not in_trade and signal == 1 and not equity_stop_signal:
            enter_price = open_price[index]
            start_price = close_price[index]
            top_price = start_price
            shares = int(inv_price / start_price) #need condition here to see if you can afford shares
            shares_arr[index] = shares
            shares_cost = enter_price * shares
            actual_enter_exit[index] = 1
            in_trade = True
            enter_index = index
        elif in_trade and ((signal == -1) or ((index - enter_index) >= run_length) or equity_stop_signal): #exit signal
            profit[enter_index] = (open_price[index] - enter_price) * shares
            exit_profit[index] = profit[enter_index]
            exit_index[enter_index] = df_index[index]
            actual_enter_exit[index] = -1
            in_trade = False
            shares = 0
        elif in_trade and ((top_price - close_price[index]) / top_price) >= stop_thresh: 
            profit[enter_index] = (open_price[index] - enter_price) * shares
            exit_profit[index] = profit[enter_index]
            exit_index[enter_index] = df_index[index]
            actual_enter_exit[index] = -1
            in_trade = False
            shares = 0
        elif index == (n - 1) and in_trade:
            profit[enter_index] = (open_price[index] - enter_price) * shares
            exit_profit[index] = profit[enter_index]
            exit_index[enter_index] = df_index[index]
            actual_enter_exit[index] = -1
            in_trade = False
            shares = 0 # lots of duplication here
        equity[index] = (shares * close_price[index]) - (shares * enter_price)
        shares_arr[index] = shares
    return profit, exit_profit, exit_index, actual_enter_exit, shares_arr, equity # don't really need exit profit here

In [17]:
df_profits1 = df_enter_exit.groupby('symbol').apply(lambda x: backtest_seq(x, stop_thresh=1.0, inv_price=10000, run_length=100))

C:\Users\K28120\py36\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [18]:
def equity_adjustment(df):
    strategy_stats_df = pd.DataFrame({'date': df['date'].unique()}).sort_values('date')
    date_equity = df.groupby('date').equity_curve.sum().reset_index('date')
    strategy_stats_df = strategy_stats_df.merge(date_equity, on='date', how='left')
    strategy_stats_df.loc[:, 'equity_curve_mv_avg'] = strategy_stats_df.set_index('date').equity_curve.rolling('200d', min_periods=1)\
                                                                       .mean().fillna(method='ffill').values
    strategy_stats_df.loc[:, 'equity_curve_agg'] = strategy_stats_df. equity_curve
    return strategy_stats_df#strategy_stats_df

In [19]:
strategy_name = 'STOROC'

In [20]:
equity_adj_df = equity_adjustment(df_profits1)

In [21]:
equity_adj_df.to_parquet('equity_curves/%s1.parquet' % strategy_name)

In [22]:
df_profits1_mrg = df_profits1.merge(equity_adj_df[['date', 'equity_curve_mv_avg', 'equity_curve_agg']], on='date', how='inner')

In [23]:
df_profits1_mrg.to_parquet('profits/%s.parquet' % strategy_name)

In [18]:
import numpy as np

def get_profit_metrics(df_profits):
    wins_losses = {}
    col_name = 'profit'
    win_index = df_profits[col_name] > 0
    loss_index = df_profits[col_name] < 0
    mean_win = df_profits.loc[win_index, col_name].mean()
    mean_loss = df_profits.loc[loss_index, col_name].mean()
    mean_norm_profit_win = df_profits.loc[win_index, 'norm_profit'].mean()
    mean_norm_profit_loss = df_profits.loc[loss_index, 'norm_profit'].mean()
    mean_norm_profit = df_profits.norm_profit.mean()
    sum_win = df_profits.loc[win_index, col_name].sum()
    sum_loss = df_profits.loc[loss_index, col_name].sum()
    
    wins_losses[col_name] = [win_index.sum(), loss_index.sum(), win_index.sum() + loss_index.sum(),
                             mean_win, mean_loss,
                             mean_norm_profit_win, mean_norm_profit_loss,
                             mean_norm_profit,
                             sum_win, sum_loss
                            ]

    df_win_loss = pd.DataFrame(wins_losses, index=['wins', 'losses', 'ttl_trades', 'mean_win',
                                                   'mean_loss',
                                                   'mean_norm_profit_win', 'mean_norm_profit_loss',
                                                   'mean_norm_profit',
                                                   'ttl_win', 'ttl_loss']).transpose()
    df_win_loss.loc[:, 'win_loss_rate'] =  df_win_loss.wins / (df_win_loss.losses + df_win_loss.wins)
    df_win_loss.loc[:, 'win_loss_ratio'] = df_win_loss.mean_win / np.abs(df_win_loss.mean_loss)
    
    df_win_loss.loc[:, 'profit_factor'] = df_win_loss.ttl_win / np.abs(df_win_loss.ttl_loss)
    df_win_loss.loc[:, 'net_profit'] = df_win_loss.ttl_win + df_win_loss.ttl_loss
    return df_win_loss

In [19]:
df_win_loss = get_profit_metrics(df_profits1)

In [ ]:
df_profits1.loc[:, 'month'] = df_profits1['date'].dt.month

In [ ]:
df_win_loss_year = df_profits1.groupby('year').apply(get_profit_metrics)

In [ ]:
df_win_loss_year_month_sym = df_profits1.groupby(['symbol', 'year', 'month']).agg({'profit': 'sum'})
df_win_loss_year_month_sym = df_win_loss_year_month_sym.loc[df_win_loss_year_month_sym.profit != 0].reset_index()

In [ ]:
prof_2008 = df_win_loss_year_month_sym.loc[df_win_loss_year_month_sym.year == 2008]

In [ ]:
prof_2008.sort_values('profit')

In [20]:
df_win_loss

,wins,losses,ttl_trades,mean_win,mean_loss,mean_norm_profit_win,mean_norm_profit_loss,mean_norm_profit,ttl_win,ttl_loss,win_loss_rate,win_loss_ratio,profit_factor,net_profit
profit,3285.0,1167.0,4452.0,1037.870048,-1464.717433,0.103884,-0.146673,0.038206,3.409403e+06,-1.709325e+06,0.737871,0.70858,1.99459,1.700078e+06


In [ ]:
df_win_loss

In [ ]:
df_win_loss

In [ ]:
df_win_loss

In [ ]:
df_win_loss

In [ ]:
df_win_loss

In [ ]:
df_win_loss

In [ ]:
df_win_loss

In [ ]:
df_win_loss_year

In [ ]:
df_win_loss_year

In [18]:
strategy_name = 'STOROC'

In [19]:
def equity_adjustment(df):
    strategy_stats_df = pd.DataFrame({'date': df['date'].unique()}).sort_values('date')
    date_prof = df.groupby('date').exit_profit.sum().reset_index('date')
    strategy_stats_df = strategy_stats_df.merge(date_prof, on='date', how='left')
    strategy_stats_df.loc[:, 'cum_prof'] = strategy_stats_df.exit_profit.cumsum()
    strategy_stats_df.loc[:, 'cum_prof_mv_avg'] = strategy_stats_df.set_index('date').cum_prof.rolling('100d', min_periods=1).mean().values
    strategy_stats_df.loc[:, 'exit_profit2'] = strategy_stats_df.exit_profit
    strategy_stats_df.loc[:, 'strategy_on'] = (strategy_stats_df.cum_prof > strategy_stats_df.cum_prof_mv_avg).fillna(False)
    strategy_stats_df.loc[~strategy_stats_df.strategy_on, 'exit_profit2'] =  0 
    strategy_stats_df.loc[:, 'cum_prof2'] = strategy_stats_df.exit_profit2.cumsum()
    return strategy_stats_df

In [20]:
equity_adj_df = equity_adjustment(df_profits1)

In [21]:
equity_adj_df.to_parquet('equity_curves/%s.parquet' % strategy_name)

In [22]:
df_profits1 = df_profits1.merge(equity_adj_df[['date', 'cum_prof', 'cum_prof_mv_avg', 'strategy_on']], on='date', how='inner')

In [23]:
df_profits1.to_parquet('profits/%s.parquet' % strategy_name)

In [24]:
df_profits1d.columns

Index(['date', 'open', 'high', 'low', 'close', 'adjusted_close', 'volume',
       'dividend_amount', 'split_coefficient', 'symbol', 'actual_split_coef',
       'open_adj', 'high_adj', 'low_adj', 'close_adj', 'year', 'join_date',
       'join_close', 'open_adj_n', 'high_adj_n', 'low_adj_n', 'close_adj_n',
       'sector', 'regime_roc', 'regime_close', 'regime_ma', 'sector_roc',
       'sector_close', 'sector_ma', 'sto', 'last_close', 'atr', 'volatility',
       'roc', 'mv_avg', 'close_adj_last', 'sto_overbought_exit', 'exits',
       'rsi_oversold_enter', 'last_sto', 'sto_roc', 'last_sto_roc',
       'sto_oversold_enter', 'entrances', 'enter_exit_sig', 'next_open',
       'profit', 'exit_profit', 'exit_date', 'cum_profit', 'purch_shares',
       'norm_profit', 'avg_profit', 'avg_profit_std', 'eavg_profit',
       'actual_enter_exit', 'actual_enter', 'actual_exit', 'trade_count',
       'cum_prof', 'cum_prof_mv_avg', 'strategy_on'],
      dtype='object')

### Improvements 
* Look for price increase after reversion based indicators - V1
* Weight examples in XGBoost by Price. Look at training vs testing profit - V2
* Add Bollinger band based indicator - V3
* Add Regime Filter - V4
* Use Symbol Performance based filtering - V5
* Look at performance across Industry